In [34]:
!pip install qiskit qiskit_aer

In [35]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_bloch_multivector
import numpy as np
import matplotlib.pyplot as plt

In [36]:
def qft_rotations(qc, n):
    """Apply QFT rotations recursively on the first n qubits."""
    if n == 0:
        return qc
    n -= 1
    qc.h(n)
    for qubit in range(n):
        qc.cp(np.pi / 2**(n - qubit), qubit, n)
    qft_rotations(qc, n)
    return qc

In [37]:
def swap_registers(qc, n):
    """Swap qubits to reverse their order."""
    for qubit in range(n // 2):
        qc.swap(qubit, n - qubit - 1)
    return qc

In [39]:
def qft_circuit(n):
    """Builds an n-qubit Quantum Fourier Transform circuit."""
    qc = QuantumCircuit(n)
    qft_rotations(qc, n)
    swap_registers(qc, n)
    qc.name = "QFT"
    return qc

In [41]:
def run_qft(n):
    """Executes QFT circuit and displays Bloch sphere representation."""
    qc = qft_circuit(n)

    # Create a simulator and save the statevector explicitly
    simulator = AerSimulator(method='statevector')
    qc.save_statevector()

    # Transpile and run
    compiled = transpile(qc, simulator)
    result = simulator.run(compiled).result()

    # Retrieve statevector safely
    statevector = result.data(0)["statevector"]

    # Display results
    plot_bloch_multivector(statevector)
    plt.show()
    print(qc.draw(output='text'))

if __name__ == "__main__":
    n_qubits = 3
    print(f"Running Quantum Fourier Transform on {n_qubits} qubits...")
    run_qft(n_qubits)

Running Quantum Fourier Transform on 3 qubits...
                                          ┌───┐    statevector 
q_0: ──────■──────────────────────■───────┤ H ├─X───────░──────
           │                ┌───┐ │P(π/2) └───┘ │       ░      
q_1: ──────┼────────■───────┤ H ├─■─────────────┼───────░──────
     ┌───┐ │P(π/4)  │P(π/2) └───┘               │       ░      
q_2: ┤ H ├─■────────■───────────────────────────X───────░──────
     └───┘                                              ░      


# **Programming Tasks**

***1. Vary the number of qubits***

Try running the QFT for 2, 3, and 4 qubits and observe how the circuit changes.



In [42]:


# Try with different numbers of qubits
for n in [2, 3, 4]:
    qc = qft_circuit(n)
    print(f"\nQFT Circuit for {n} qubits:")
    print(qc.draw(output='text'))



QFT Circuit for 2 qubits:
                   ┌───┐   
q_0: ──────■───────┤ H ├─X─
     ┌───┐ │P(π/2) └───┘ │ 
q_1: ┤ H ├─■─────────────X─
     └───┘                 

QFT Circuit for 3 qubits:
                                          ┌───┐   
q_0: ──────■──────────────────────■───────┤ H ├─X─
           │                ┌───┐ │P(π/2) └───┘ │ 
q_1: ──────┼────────■───────┤ H ├─■─────────────┼─
     ┌───┐ │P(π/4)  │P(π/2) └───┘               │ 
q_2: ┤ H ├─■────────■───────────────────────────X─
     └───┘                                        

QFT Circuit for 4 qubits:
                                                                          ┌───┐»
q_0: ──────■───────────────────────────────■──────────────────────■───────┤ H ├»
           │                               │                ┌───┐ │P(π/2) └───┘»
q_1: ──────┼────────■──────────────────────┼────────■───────┤ H ├─■─────────X──»
           │        │                ┌───┐ │P(π/4)  │P(π/2) └───┘           │  »
q_2: ──────┼─────

***2. Inverse QFT***

Modify the code to implement the inverse QFT (apply inverse rotations and swap again).



In [43]:
def inverse_qft(qc, n):
    for qubit in range(n // 2):
        qc.swap(qubit, n - qubit - 1)
    for j in reversed(range(n)):
        for m in reversed(range(j)):
            qc.cp(-np.pi / 2**(j - m), m, j)
        qc.h(j)
    return qc

def inverse_qft_circuit(n):
    qc = QuantumCircuit(n)
    inverse_qft(qc, n)
    qc.name = "QFT†"
    return qc

qc_inv = inverse_qft_circuit(3)
print(qc_inv.draw(output='text'))


                                      ┌───┐
q_0: ─X────────────■─────────■────────┤ H ├
      │            │         │P(-π/2) ├───┤
q_1: ─┼──■─────────┼─────────■────────┤ H ├
      │  │P(-π/2)  │P(-π/4)   ┌───┐   └───┘
q_2: ─X──■─────────■──────────┤ H ├────────
                              └───┘        


***3. Integration with Phase Estimation***

Combine QFT with a simple phase estimation circuit.

In [44]:
from qiskit import QuantumCircuit
import numpy as np

def phase_estimation(num_counting_qubits=3, phase=0.375):
    n = num_counting_qubits
    qc = QuantumCircuit(n + 1, n)

    # Initialize
    qc.h(range(n))
    qc.x(n)  # target qubit

    # Apply controlled unitary rotations (phase shift)
    for i in range(n):
        qc.cp(2 * np.pi * phase * 2**i, i, n)

    # Apply inverse QFT to counting register
    inverse_qft(qc, n)
    qc.measure(range(n), range(n))

    return qc

qc_pe = phase_estimation()
print(qc_pe.draw(output='text'))

# Run
sim = AerSimulator()
result = sim.run(transpile(qc_pe, sim), shots=1024).result()
counts = result.get_counts()
print(counts)


     ┌───┐                                                             ┌───┐┌─┐»
q_0: ┤ H ├─■───────────────────────────X────────────■─────────■────────┤ H ├┤M├»
     ├───┤ │                           │            │         │P(-π/2) ├───┤└╥┘»
q_1: ┤ H ├─┼─────────■─────────────────┼──■─────────┼─────────■────────┤ H ├─╫─»
     ├───┤ │         │                 │  │P(-π/2)  │P(-π/4)   ┌───┐   └┬─┬┘ ║ »
q_2: ┤ H ├─┼─────────┼─────────■───────X──■─────────■──────────┤ H ├────┤M├──╫─»
     ├───┤ │P(3π/4)  │P(3π/2)  │P(3π)                          └───┘    └╥┘  ║ »
q_3: ┤ X ├─■─────────■─────────■─────────────────────────────────────────╫───╫─»
     └───┘                                                               ║   ║ »
c: 3/════════════════════════════════════════════════════════════════════╩═══╩═»
                                                                         2   0 »
«        
«q_0: ───
«     ┌─┐
«q_1: ┤M├
«     └╥┘
«q_2: ─╫─
«      ║ 
«q_3: ─╫─
«      ║ 
«c: 3/═╩═
«      1 

***4. Measure Output States***

Add measurements and simulate the probability distribution of different states.

In [45]:
def qft_with_measurement(n):
    qc = QuantumCircuit(n, n)
    qft_rotations(qc, n)
    swap_registers(qc, n)
    qc.measure(range(n), range(n))
    return qc

sim = AerSimulator()
qc = qft_with_measurement(3)
result = sim.run(transpile(qc, sim), shots=1024).result()
counts = result.get_counts()

from qiskit.visualization import plot_histogram
print(counts)
plot_histogram(counts)
plt.show()


{'110': 137, '001': 123, '101': 137, '000': 150, '010': 116, '100': 126, '011': 126, '111': 109}


***5. Circuit Visualization***

Use qc.draw('mpl') to plot the circuit diagram.



In [46]:
qc = qft_circuit(3)
qc.draw()  # Shows the graphical circuit layout


┌───┐   
q_0: ──────■──────────────────────■───────┤ H ├─X─
           │                ┌───┐ │P(π/2) └───┘ │ 
q_1: ──────┼────────■───────┤ H ├─■─────────────┼─
     ┌───┐ │P(π/4)  │P(π/2) └───┘               │ 
q_2: ┤ H ├─■────────■───────────────────────────X─
     └───┘